In [1]:
import pandas as pd
import numpy as np

from plotly import graph_objects as go
from plotly import express as px
from plotly.subplots import make_subplots

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# Import data from CSV file

In [2]:
df = pd.read_csv(
    "data/dailyATM.csv",
    parse_dates=["timestamp"],
    index_col="timestamp"
)

# make sure index is datetime
df.index = pd.to_datetime(df.index)

# enforce daily frequency (fills missing days with NaN)
df = df.asfreq("D")

df.head(20)

,arrivals,departures
timestamp,,
2023-12-31 00:00:00+00:00,0,1
2024-01-01 00:00:00+00:00,121,122
2024-01-02 00:00:00+00:00,119,119
2024-01-03 00:00:00+00:00,121,119
2024-01-04 00:00:00+00:00,127,127
2024-01-05 00:00:00+00:00,137,137
2024-01-06 00:00:00+00:00,139,139
2024-01-07 00:00:00+00:00,124,123
2024-01-08 00:00:00+00:00,114,111


In [3]:
fig = go.Figure()


fig.add_trace(
    go.Scatter(
        x=df.index,
        y=df["arrivals"],
        mode="lines",
        name="Arrivals",
    )
)

fig.add_trace(
    go.Scatter(
        x=df.index,
        y=df["departures"],
        mode="lines",
        name="Departures",
    )
)

fig.update_layout(
    height=600,
    width=1000,
    xaxis_title="Time",
    yaxis_title="Daily Movements",
)

fig.show()

fig.write_image("daily_movements.pdf")

# Resample data to weekly

In [ ]:
df_weekly = df.resample("W").sum()
df_weekly.head(10)

,arrivals,departures
timestamp,,
2023-12-31 00:00:00+00:00,0,1
2024-01-07 00:00:00+00:00,888,886
2024-01-14 00:00:00+00:00,710,705
2024-01-21 00:00:00+00:00,687,685
2024-01-28 00:00:00+00:00,682,667
2024-02-04 00:00:00+00:00,683,682
2024-02-11 00:00:00+00:00,739,729
2024-02-18 00:00:00+00:00,789,782
2024-02-25 00:00:00+00:00,780,782


In [8]:
df_weekly_avg = df.resample("W").mean()
df_weekly_avg.head(10)

,arrivals,departures
timestamp,,
2023-12-31 00:00:00+00:00,0.000000,1.000000
2024-01-07 00:00:00+00:00,126.857143,126.571429
2024-01-14 00:00:00+00:00,101.428571,100.714286
2024-01-21 00:00:00+00:00,98.142857,97.857143
2024-01-28 00:00:00+00:00,97.428571,95.285714
2024-02-04 00:00:00+00:00,97.571429,97.428571
2024-02-11 00:00:00+00:00,105.571429,104.142857
2024-02-18 00:00:00+00:00,112.714286,111.714286
2024-02-25 00:00:00+00:00,111.428571,111.714286


# Decomposition

In [ ]:
decompose = seasonal_decompose(df["departures"], model="additive", period=24)


# Create a 4-row subplot figure
fig = make_subplots(rows=4, cols=1, shared_xaxes=True, 
                    subplot_titles=["Observed", "Trend", "Seasonal", "Residual"])

# Add traces for each component
fig.add_trace(go.Scatter(x=decompose.observed.index, y=decompose.observed, 
                         mode='lines', line=dict(color='black', width=2), name="Observed"), row=1, col=1)

fig.add_trace(go.Scatter(x=decompose.trend.index, y=decompose.trend, 
                         mode='lines', line=dict(color='blue', width=2), name="Trend"), row=2, col=1)

fig.add_trace(go.Scatter(x=decompose.seasonal.index, y=decompose.seasonal, 
                         mode='lines', line=dict(color='green', width=2), name="Seasonal"), row=3, col=1)

fig.add_trace(go.Scatter(x=decompose.resid.index, y=decompose.resid, 
                         mode='lines', line=dict(color='cyan', width=2), name="Residual"), row=4, col=1)

# Update layout
fig.update_layout(
    height=800, width=1000,
    showlegend=False,
    title_text="Time Series Decomposition",
    xaxis4_title="Date"
)

# Enable grid lines
fig.update_xaxes(showgrid=True)
fig.update_yaxes(showgrid=True)

fig.show()

# Basic Forecasting Methods

In [5]:
# Forecast Horizon of 30 days
forecast_horizon = 30

# Create a forecast index
forecast_index = pd.date_range(start=df.index[-1] + pd.Timedelta(days=1), 
                               periods=forecast_horizon, 
                               freq='D')

# Naive Method
forecast_naive = [df['arrivals'].iloc[-1]] * forecast_horizon
forecast_naive = pd.Series(forecast_naive, index=forecast_index)

# Average Method
forecast_mean = [df['arrivals'].mean()] * forecast_horizon
forecast_mean = pd.Series(forecast_mean, index=forecast_index)

# Drift Method
y_T = df['arrivals'].iloc[-1]  # Last observed value
y_1 = df['arrivals'].iloc[0]   # First observed value
T = len(df['arrivals'])        # Number of observations
drift = y_T + (y_T - y_1) / (T - 1) * np.arange(1, forecast_horizon + 1)
drift_forecast = pd.Series(drift, index=forecast_index)


# # Combine the original data with the forecast
# df_combined = pd.concat([df["arrivals"], forecast_naive])

# Plot the combined data
fig = go.Figure()

# Add historical data
fig.add_trace(go.Scatter(x=df.index, y=df["arrivals"],
                         mode="lines+markers", name="Observations", line=dict(color='blue')))

# Add forecast data
fig.add_trace(go.Scatter(x=forecast_mean.index, y=forecast_naive,
                         mode="lines+markers", name="Naive Forecast", line=dict(color='red', dash="dash")))

fig.add_trace(go.Scatter(x=forecast_mean.index, y=forecast_mean,
                         mode="lines+markers", name="Mean Forecast", line=dict(color='green', dash="dash")))

fig.add_trace(go.Scatter(x=drift_forecast.index, y=drift_forecast,
                         mode="lines+markers", name="Drift Forecast", line=dict(color='violet', dash="dash")))

# Update layout
fig.update_layout(
    title="Number of Daily Arrivals with Naive Forecast",
    xaxis_title="Time",
    yaxis_title="Number of Arrivals per Day",
    font=dict(size=16),
    showlegend=True
)

fig.show()

# Simple Exponential Smoothing

In [6]:
df['arrivals_change']=df['arrivals'].diff()

# Create interactive plot
fig = px.line(df, x=df.index, y="arrivals_change", markers=True, 
              title="Daily Change in Number of Arrivals")

# Update layout
fig.update_layout(
    xaxis_title="Date",
    yaxis_title="Daily Change in Number of Arrivals",
    font=dict(size=16),
    showlegend=True
)

fig.show()

In [31]:

alpha = 0.3
model = SimpleExpSmoothing(df['arrivals_change'].dropna(), initialization_method="heuristic")
fitted_model = model.fit(smoothing_level=alpha, optimized=False)


# Forecast the next 30 days
forecast_horizon = 30
forecast = fitted_model.forecast(forecast_horizon)

# Create a forecast index
forecast_index = pd.date_range(start=df.index[-1] + pd.Timedelta(days=1), 
                               periods=forecast_horizon, 
                               freq='D')

# Convert forecast to a Pandas Series
forecast_series = pd.Series(forecast, index=forecast_index)


fig = go.Figure()

# Observations
fig.add_trace(
    go.Scatter(
        x=df.index,
        y=df["arrivals_change"],
        mode="lines+markers",
        name="Observations",
        line=dict(color="blue", width=2),
        marker=dict(size=6),
    )
)

# Fitted values
fig.add_trace(
    go.Scatter(
        x=fitted_model.fittedvalues.index,
        y=fitted_model.fittedvalues,
        mode="lines",
        name="Fitted Values",
        line=dict(color="black", width=2, dash="dash"),
    )
)

# Forecast
fig.add_trace(
    go.Scatter(
        x=forecast_series.index,
        y=forecast_series,
        mode="lines",
        name="Forecast",
        line=dict(color="red", width=2),
    )
)

fig.update_layout(
    height=600,
    width=1000,
    xaxis_title="Time",
    yaxis_title="Daily Change Arrivals",
    title=rf"Exponential Smoothing, $\alpha = {alpha}$",
    font=dict(size=16),
    legend=dict(font_size=14),
)

fig.update_xaxes(showgrid=True)
fig.update_yaxes(showgrid=True)

fig.show()

# Exponential Smoothing with Trend

In [10]:
df_filtered = df.loc["2024-01-08":"2024-03-23"]

# Plot the combined data
fig = go.Figure()

# Add historical data
fig.add_trace(go.Scatter(x=df_filtered.index, y=df_filtered['arrivals'],
                         mode="lines+markers", name="Observations", line=dict(color='blue')))

In [ ]:

alpha = 0.7
beta = 0.005

# Fit the model with additive trend and multiplicative seasonality
model = ExponentialSmoothing(
    df_filtered['arrivals'],
    trend="add",
    seasonal=None,
    initialization_method="known",  # Use known initialization
    initial_level=df_filtered['arrivals'].iloc[0],  # Provide initial level directly
    initial_trend=0                    # Provide initial trend directly
)


# Fit the model with specified smoothing parameters
fitted_model = model.fit(
      smoothing_level=alpha,    # Alpha: Level smoothing
      smoothing_trend=beta,     # Beta: Trend smoothing (Replaces smoothing_slope)
      optimized=False           # Set to False since we specify parameters
)

# Forecast for the next 14 days
forecast_horizon = 14
forecast = fitted_model.forecast(forecast_horizon)

# Create a forecast index
forecast_index = pd.date_range(start=df_filtered['arrivals'].index[-1] + pd.Timedelta(days=1),
                               periods=forecast_horizon, 
                               freq='D')

# Convert the forecast to a Pandas Series
forecast_series = pd.Series(forecast, index=forecast_index)

# Plot the combined data
fig = go.Figure()

# Add historical data
fig.add_trace(go.Scatter(x=df_filtered.index, y=df_filtered['arrivals'],
                         mode="lines+markers", name="Observations", line=dict(color='blue')))

# Add forecast data
fig.add_trace(go.Scatter(x=forecast_series.index, y=forecast_series,
                         mode="lines+markers", name="Forecast", line=dict(color='red')))

# Add the ground truth for the forecast period
df_ground_truth = df.loc["2024-03-24":"2024-04-26"]
fig.add_trace(go.Scatter(x=df_ground_truth.index, y=df_ground_truth['arrivals'],
                         mode="lines+markers", name="Ground Truth", line=dict(color='black', dash="dash")))

# Update layout
fig.update_layout(
    title_text=f"Exponential Smoothing on Daily Arrivals, α={alpha:.2f}, β={beta:.2e}",
    xaxis_title="Time",
    yaxis_title="Daily Arrivals",
    font=dict(size=16),
    showlegend=True
)

fig.show()

# Exponential Smoothing with Trend and Seasonality

In [29]:
alpha = 0.31
beta = 1.37e-09
gamma = 0.60

# Apply Holt-Winters' Exponential Smoothing
model = ExponentialSmoothing(
    df_filtered['arrivals'],
    trend="add",
    seasonal="add",          # safer than "mul" if zeros exist
    seasonal_periods=7,      # weekly seasonality
    initialization_method="estimated"
)

# Fit the model
fitted_model = model.fit(
    smoothing_level=alpha,      # Alpha: Level smoothing
    smoothing_trend=beta,       # Beta: Trend smoothing
    smoothing_seasonal=gamma,   # Gamma: Seasonal smoothing
    optimized=False             # Set to False since we specify parameters
)

# Forecast for the next 2 years (24 months)
forecast_horizon = 14
forecast = fitted_model.forecast(forecast_horizon)

# Create a forecast index
forecast_index = pd.date_range(
    start=df_filtered.index[-1] + pd.Timedelta(days=1),
    periods=forecast_horizon,
    freq="D"
)

# Convert the forecast to a Pandas Series
forecast_series = pd.Series(forecast, index=forecast_index)

# Plot the combined data
fig = go.Figure()

# Add historical data
fig.add_trace(go.Scatter(x=df_filtered.index, y=df_filtered['arrivals'],
                         mode="lines+markers", name="Observations", line=dict(color='blue')))

# Add forecast data
fig.add_trace(
    go.Scatter(
        x=forecast_series.index,
        y=forecast_series,
        mode="lines+markers",
        name="Forecast",
        line=dict(color="red")
    )
)

# Add the ground truth for the forecast period
df_ground_truth = df.loc["2024-03-24":"2024-04-26"]
fig.add_trace(go.Scatter(x=df_ground_truth.index, y=df_ground_truth['arrivals'],
                         mode="lines+markers", name="Ground Truth", line=dict(color='black', dash="dash")))

# Update layout
fig.update_layout(
    title=f'Exponential Smoothing with Trend and Seasonality, α={alpha}, β={beta}',
    xaxis_title="Year",
    yaxis_title="Daily Arrivals",
    font=dict(size=16),
    showlegend=True
)

fig.show()

In [ ]:


alpha = 0.31
beta = 1.37e-09
gamma = 0.60

# Define the series explicitly
y = df_filtered["arrivals"].asfreq("D")

# (optional but recommended) handle missing days
y = y.interpolate()   # or y = y.dropna()

# Apply Holt-Winters' Exponential Smoothing
model = ExponentialSmoothing(
    y,
    trend="add",
    seasonal="add",
    seasonal_periods=7,
    initialization_method="estimated"
)

# Fit the model
fitted_model = model.fit(
    smoothing_level=alpha,
    smoothing_trend=beta,
    smoothing_seasonal=gamma,
    optimized=False
)

# Forecast for the next 14 days
forecast_horizon = 14
forecast = fitted_model.forecast(forecast_horizon)

# Create a forecast index starting the next day
forecast_index = pd.date_range(
    start=y.index[-1] + pd.Timedelta(days=1),
    periods=forecast_horizon,
    freq="D"
)

# Convert forecast to a Series
forecast_series = pd.Series(forecast.values, index=forecast_index)

In [ ]:
# Plot the combined data
fig = go.Figure()

# Add historical data
fig.add_trace(go.Scatter(x=df_filtered.index, y=df_filtered['arrivals'],
                         mode="lines+markers", name="Observations", line=dict(color='blue')))

# Add forecast data
fig.add_trace(go.Scatter(x=forecast_series.index, y=forecast_series,
                         mode="lines+markers", name="Forecast", line=dict(color='red')))

# Add the ground truth for the forecast period
df_ground_truth = df.loc["2024-03-24":"2024-04-26"]
fig.add_trace(go.Scatter(x=df_ground_truth.index, y=df_ground_truth['arrivals'],
                         mode="lines+markers", name="Ground Truth", line=dict(color='black', dash="dash")))


# Update layout
fig.update_layout(
    title=f'Exponential Smoothing with Trend and Seasonality, α={alpha}, β={beta}',
    xaxis_title="Year",
    yaxis_title="Daily Arrivals",
    font=dict(size=16),
    showlegend=True
)

fig.show()